# STOCK CLOSING PRICE PREDICTION

In [1]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas_datareader as pdr

key="30c0973b8a648106ae38faea9031b9a3924c7469"

In [2]:
#Getting Dataset
df=pdr.get_data_tiingo('DIS', api_key=key)

C:\Users\sneha\AppData\Local\Temp\ipykernel_30900\3742500115.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df=pdr.get_data_tiingo('DIS', api_key=key)


In [3]:
df.to_csv('DIS.csv')

In [4]:
df=pd.read_csv('DIS.csv', date_parser=True)

In [5]:
df.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-21 00:00:00+00:00,113.03,113.985,112.30,112.50,7242467,110.794052,111.730160,110.078492,110.274536,7242467,0.0,1.0
1,DIS,2018-11-23 00:00:00+00:00,112.08,112.790,111.74,112.24,3242322,109.862844,110.558799,109.529570,110.019679,3242322,0.0,1.0
2,DIS,2018-11-26 00:00:00+00:00,112.55,113.180,111.92,113.18,7002353,110.323547,110.941084,109.706009,110.941084,7002353,0.0,1.0
3,DIS,2018-11-27 00:00:00+00:00,113.90,114.100,112.41,113.61,7357599,111.646841,111.842885,110.186316,111.362578,7357599,0.0,1.0
4,DIS,2018-11-28 00:00:00+00:00,116.10,116.300,114.05,114.32,9039821,113.803321,113.999365,111.793874,112.058533,9039821,0.0,1.0


In [6]:
df.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
1251,DIS,2023-11-13 00:00:00+00:00,89.44,89.9000,87.4051,88.42,12834443,89.44,89.9000,87.4051,88.42,12834443,0.0,1.0
1252,DIS,2023-11-14 00:00:00+00:00,91.07,91.5417,89.9900,90.03,13152029,91.07,91.5417,89.9900,90.03,13152029,0.0,1.0
1253,DIS,2023-11-15 00:00:00+00:00,93.93,94.5650,92.7400,92.99,20593054,93.93,94.5650,92.7400,92.99,20593054,0.0,1.0
1254,DIS,2023-11-16 00:00:00+00:00,94.51,94.6100,93.6300,93.89,12923919,94.51,94.6100,93.6300,93.89,12923919,0.0,1.0
1255,DIS,2023-11-17 00:00:00+00:00,94.15,95.3770,93.9200,94.75,12201304,94.15,95.3770,93.9200,94.75,12201304,0.0,1.0


## Data Pre-processing

In [7]:
df['date'] = pd.to_datetime(df['date'])
# df['date'] = df['date'].dt.date
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-21 00:00:00+00:00,113.03,113.9850,112.3000,112.50,7242467,110.794052,111.730160,110.078492,110.274536,7242467,0.0,1.0
1,DIS,2018-11-23 00:00:00+00:00,112.08,112.7900,111.7400,112.24,3242322,109.862844,110.558799,109.529570,110.019679,3242322,0.0,1.0
2,DIS,2018-11-26 00:00:00+00:00,112.55,113.1800,111.9200,113.18,7002353,110.323547,110.941084,109.706009,110.941084,7002353,0.0,1.0
3,DIS,2018-11-27 00:00:00+00:00,113.90,114.1000,112.4100,113.61,7357599,111.646841,111.842885,110.186316,111.362578,7357599,0.0,1.0
4,DIS,2018-11-28 00:00:00+00:00,116.10,116.3000,114.0500,114.32,9039821,113.803321,113.999365,111.793874,112.058533,9039821,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,DIS,2023-11-13 00:00:00+00:00,89.44,89.9000,87.4051,88.42,12834443,89.440000,89.900000,87.405100,88.420000,12834443,0.0,1.0
1252,DIS,2023-11-14 00:00:00+00:00,91.07,91.5417,89.9900,90.03,13152029,91.070000,91.541700,89.990000,90.030000,13152029,0.0,1.0
1253,DIS,2023-11-15 00:00:00+00:00,93.93,94.5650,92.7400,92.99,20593054,93.930000,94.565000,92.740000,92.990000,20593054,0.0,1.0
1254,DIS,2023-11-16 00:00:00+00:00,94.51,94.6100,93.6300,93.89,12923919,94.510000,94.610000,93.630000,93.890000,12923919,0.0,1.0


In [8]:
# Splitting train and test set
df_training=df[df['date']<='2022-01-01'].copy()
df_test=df[df['date']>'2022-01-01'].copy()
df_test, df_training

(     symbol                      date   close      high       low    open  \
 783     DIS 2022-01-03 00:00:00+00:00  156.76  157.5600  155.3600  155.83   
 784     DIS 2022-01-04 00:00:00+00:00  155.73  160.3200  155.5500  158.59   
 785     DIS 2022-01-05 00:00:00+00:00  155.19  159.3800  155.1000  156.52   
 786     DIS 2022-01-06 00:00:00+00:00  156.90  157.7700  153.6791  156.24   
 787     DIS 2022-01-07 00:00:00+00:00  157.83  159.3000  156.2900  156.90   
 ...     ...                       ...     ...       ...       ...     ...   
 1251    DIS 2023-11-13 00:00:00+00:00   89.44   89.9000   87.4051   88.42   
 1252    DIS 2023-11-14 00:00:00+00:00   91.07   91.5417   89.9900   90.03   
 1253    DIS 2023-11-15 00:00:00+00:00   93.93   94.5650   92.7400   92.99   
 1254    DIS 2023-11-16 00:00:00+00:00   94.51   94.6100   93.6300   93.89   
 1255    DIS 2023-11-17 00:00:00+00:00   94.15   95.3770   93.9200   94.75   
 
         volume  adjClose   adjHigh    adjLow  adjOpen  adjVol

In [9]:
training_df=df_training.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
training_df

,close,high,low,open,volume
0,113.03,113.985,112.300,112.50,7242467
1,112.08,112.790,111.740,112.24,3242322
2,112.55,113.180,111.920,113.18,7002353
3,113.90,114.100,112.410,113.61,7357599
4,116.10,116.300,114.050,114.32,9039821
...,...,...,...,...,...
778,152.80,154.020,151.400,152.88,7765470
779,155.20,156.500,152.410,152.62,12198679
780,154.87,155.920,154.180,154.56,8737004
781,155.93,157.070,155.515,155.71,7228438


## Feature Scaling

In [10]:
scaler=MinMaxScaler(feature_range=(0,1))
training_df=scaler.fit_transform(training_df)
training_df

array([[0.23478261, 0.23073268, 0.28565288, 0.24210208, 0.05338903],
       [0.22660353, 0.22040781, 0.28083899, 0.23985479, 0.00615609],
       [0.23065002, 0.22377743, 0.28238631, 0.2479796 , 0.05055381],
       ...,
       [0.59500646, 0.5930534 , 0.6456632 , 0.60564415, 0.07103623],
       [0.60413259, 0.60298946, 0.65713917, 0.61558408, 0.05322338],
       [0.59517865, 0.59866943, 0.65047709, 0.61696703, 0.04096006]])

## Converting into Dataset Matrix (taking previous 60 values)

In [11]:
X_train=[]
y_train=[]

In [12]:
training_df.shape

(783, 5)

In [13]:
ltrain=training_df.shape[0]

for i in range(60, ltrain):
    X_train.append(training_df[i-60:i])
    y_train.append(training_df[i,0])

In [14]:
X_train, y_train=np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((723, 60, 5), (723,))

## Building LSTM 

In [15]:
regressior=Sequential()
regressior.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=80, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=120, activation='relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units=1))

regressior.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 60)            15840     
                                                                 
 dropout (Dropout)           (None, 60, 60)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 60)            29040     
                                                                 
 dropout_1 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 80)            45120     
                                                                 
 dropout_2 (Dropout)         (None, 60, 80)            0         
                                                                 
 lstm_3 (LSTM)               (None, 120)               9

In [16]:
regressior.compile(optimizer='adam', loss='mean_squared_error')
regressior.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
12/12 [==============================] - 20s 327ms/step - loss: 0.1329
Epoch 2/50
12/12 [==============================] - 5s 395ms/step - loss: 0.0193
Epoch 3/50
12/12 [==============================] - 4s 314ms/step - loss: 0.0134
Epoch 4/50
12/12 [==============================] - 5s 403ms/step - loss: 0.0112
Epoch 5/50
12/12 [==============================] - 6s 516ms/step - loss: 0.0108
Epoch 6/50
12/12 [==============================] - 9s 755ms/step - loss: 0.0095
Epoch 7/50
12/12 [==============================] - 6s 523ms/step - loss: 0.0099
Epoch 8/50
12/12 [==============================] - 7s 557ms/step - loss: 0.0091
Epoch 9/50
12/12 [==============================] - 6s 491ms/step - loss: 0.0082
Epoch 10/50
12/12 [==============================] - 8s 687ms/step - loss: 0.0079
Epoch 11/50
12/12 [==============================] - 5s 412ms/step - loss: 0.0076
Epoch 12/50
12/12 [==============================] - 5s 441ms/step - loss: 0.0075
Epoch 13/50
12/12 [=====

## Preparing Test Dataset

In [17]:
df_test.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
783,DIS,2022-01-03 00:00:00+00:00,156.76,157.56,155.3600,155.83,10228257,156.76,157.56,155.3600,155.83,10228257,0.0,1.0
784,DIS,2022-01-04 00:00:00+00:00,155.73,160.32,155.5500,158.59,16582041,155.73,160.32,155.5500,158.59,16582041,0.0,1.0
785,DIS,2022-01-05 00:00:00+00:00,155.19,159.38,155.1000,156.52,12272097,155.19,159.38,155.1000,156.52,12272097,0.0,1.0
786,DIS,2022-01-06 00:00:00+00:00,156.90,157.77,153.6791,156.24,11095326,156.90,157.77,153.6791,156.24,11095326,0.0,1.0
787,DIS,2022-01-07 00:00:00+00:00,157.83,159.30,156.2900,156.90,9554622,157.83,159.30,156.2900,156.90,9554622,0.0,1.0


In [18]:
df_training.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
778,DIS,2021-12-27 00:00:00+00:00,152.80,154.02,151.400,152.88,7765470,152.80,154.02,151.400,152.88,7765470,0.0,1.0
779,DIS,2021-12-28 00:00:00+00:00,155.20,156.50,152.410,152.62,12198679,155.20,156.50,152.410,152.62,12198679,0.0,1.0
780,DIS,2021-12-29 00:00:00+00:00,154.87,155.92,154.180,154.56,8737004,154.87,155.92,154.180,154.56,8737004,0.0,1.0
781,DIS,2021-12-30 00:00:00+00:00,155.93,157.07,155.515,155.71,7228438,155.93,157.07,155.515,155.71,7228438,0.0,1.0
782,DIS,2021-12-31 00:00:00+00:00,154.89,156.57,154.740,155.87,6189861,154.89,156.57,154.740,155.87,6189861,0.0,1.0


In [19]:
past_60_days=df_training.tail(60)
# past_60_days
df1=past_60_days.append(df_test) #past 60 days taken for the first value of test data
df1=df1.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
df1

C:\Users\sneha\AppData\Local\Temp\ipykernel_30900\1121012054.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1=past_60_days.append(df_test) #past 60 days taken for the first value of test data


,close,high,low,open,volume
723,177.71,179.6300,177.3300,177.70,7779997
724,176.74,178.6000,176.4029,177.93,4132182
725,173.52,176.5200,173.4800,176.02,8046270
726,173.13,174.4900,172.5450,173.62,5188380
727,172.96,173.7000,170.9400,173.63,8918152
...,...,...,...,...,...
1251,89.44,89.9000,87.4051,88.42,12834443
1252,91.07,91.5417,89.9900,90.03,13152029
1253,93.93,94.5650,92.7400,92.99,20593054
1254,94.51,94.6100,93.6300,93.89,12923919


In [20]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(df1)
df1

# inputs=scaler.transform(df1)
# inputs

array([[1.        , 1.        , 1.        , 0.99005924, 0.05515183],
       [0.99014127, 0.98968308, 0.99059728, 0.99236871, 0.        ],
       [0.95741437, 0.96884891, 0.96095299, 0.97319008, 0.05917765],
       ...,
       [0.1484907 , 0.14795264, 0.14208142, 0.13947183, 0.24887423],
       [0.15438561, 0.14840338, 0.15110787, 0.14850889, 0.1329235 ],
       [0.1507267 , 0.15608598, 0.15404907, 0.15714429, 0.12199819]])

In [21]:
df1.shape

(533, 5)

In [22]:
X_test=[]
y_test=[]

In [23]:
ltest=df1.shape[0]
for i in range(60,ltest):
    X_test.append(df1[i-60:i])
    y_test.append(df1[i,0])

In [24]:
X_test, y_test=np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((473, 60, 5), (473,))

## Predicting the stock closing price

In [25]:
y_pred=regressior.predict(X_test)

15/15 [==============================] - 3s 89ms/step


In [26]:
y_test

array([7.87071857e-01, 7.76603313e-01, 7.71114951e-01, 7.88494766e-01,
       7.97946946e-01, 7.85445675e-01, 7.98556764e-01, 7.97642037e-01,
       7.73655859e-01, 7.38083139e-01, 7.41437138e-01, 7.19483687e-01,
       6.94176237e-01, 5.90100620e-01, 5.90913711e-01, 5.81258258e-01,
       5.51682082e-01, 5.70179896e-01, 6.02805163e-01, 6.46915337e-01,
       6.62364061e-01, 6.43358065e-01, 6.17034251e-01, 6.37259884e-01,
       6.42240065e-01, 6.41935156e-01, 6.90212420e-01, 7.40319138e-01,
       7.12978961e-01, 7.27004777e-01, 7.66338042e-01, 7.82904767e-01,
       7.48348409e-01, 7.32188231e-01, 6.98953146e-01, 6.76389877e-01,
       7.12267507e-01, 7.13588779e-01, 7.02713690e-01, 6.74662059e-01,
       6.91330420e-01, 6.73340787e-01, 6.24047159e-01, 5.50665718e-01,
       5.32879358e-01, 5.52190263e-01, 5.52088627e-01, 5.32879358e-01,
       5.05234272e-01, 5.57780262e-01, 5.97824982e-01, 6.11342616e-01,
       6.19778433e-01, 6.02703527e-01, 6.17847342e-01, 5.92743165e-01,
      

## Visualization the results

In [27]:
import plotly.graph_objs as go

# Create a trace for actual values
trace_actual = go.Scatter(x=df_test['date'], y=y_test, mode='lines', name='Actual', line=dict(color='red'))

# Create a trace for predicted values
trace_pred = go.Scatter(x=df_test['date'], y=y_pred.flatten(), mode='lines', name='Predicted', line=dict(color='green'))

# Create a trace for markers with reduced size
trace_markers1 = go.Scatter(
    x=df_test['date'],
    y=y_test.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='red', size=3),  # Adjust the size here
)

trace_markers2 = go.Scatter(
    x=df_test['date'],
    y=y_pred.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='green', size=3),  # Adjust the size here
)

# Create the Plotly figure
fig = go.Figure(data=[trace_actual, trace_pred, trace_markers1, trace_markers2])

# Customize the layout
fig.update_layout(
    title='Stock Price Prediction',
    xaxis_title='Time',
    yaxis_title='Stock Price',
     width=800,
    height=600
)

# Show the Plotly figure
fig.show()


In [ ]:
df_past_60_days_for_future = df_test.tail(60).copy()
df_past_60_days_for_future.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
df_past_60_days_for_future

future_stock_value=[]

n=df_past_60_days_for_future.shape[0]
for i in range(0,n):
    future_stock_value.append(df_past_60_days_for_future[i:0])

y_pred=regressior.predict(future_stock_value)